<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/068_Agent_to_Agent_Communication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🤝 Agent-to-Agent Communication

Imagine building a system where multiple specialized agents collaborate — each contributing their own domain expertise to solve complex problems together.

For instance:

* A **primary agent** coordinates the high-level strategy.
* Meanwhile, **specialist agents** are delegated specific tasks like scheduling, compliance checking, or summarization.

To enable this teamwork, agents must **communicate with each other**.

---

### 🔧 Exposing Capabilities as Tools

One of the simplest and most effective ways to empower agent communication is to **expose another agent’s functionality as a tool**.

By doing this:

* The primary agent doesn’t need to know the internal workings of the specialist.
* Coordination becomes modular, scalable, and reusable.

Here's the architectural beauty:

> 🧠 **Each agent remains focused and self-contained**, but collaboration is made possible via a clean and simple tool interface.

This tool is the **gateway to multi-agent collaboration**, providing a safe and structured mechanism for one agent to delegate to another — with memory isolation and context control built-in.


In [ ]:
@register_tool()
def call_agent(action_context: ActionContext,
               agent_name: str,
               task: str) -> dict:
    """
    Invoke another agent to perform a specific task.

    Args:
        action_context: Contains registry of available agents
        agent_name: Name of the agent to call
        task: The task to ask the agent to perform

    Returns:
        The result from the invoked agent's final memory
    """
    # Get the agent registry from our context
    agent_registry = action_context.get_agent_registry()
    if not agent_registry:
        raise ValueError("No agent registry found in context")

    # Get the agent's run function from the registry
    agent_run = agent_registry.get_agent(agent_name)
    if not agent_run:
        raise ValueError(f"Agent '{agent_name}' not found in registry")

    # Create a new memory instance for the invoked agent
    invoked_memory = Memory()

    try:
        # Run the agent with the provided task
        result_memory = agent_run(
            user_input=task,
            memory=invoked_memory,
            # Pass through any needed context properties
            action_context_props={
                'auth_token': action_context.get('auth_token'),
                'user_config': action_context.get('user_config'),
                # Don't pass agent_registry to prevent infinite recursion
            }
        )

        # Get the last memory item as the result
        if result_memory.items:
            last_memory = result_memory.items[-1]
            return {
                "success": True,
                "agent": agent_name,
                "result": last_memory.get("content", "No result content")
            }
        else:
            return {
                "success": False,
                "error": "Agent failed to run."
            }

    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

This code is a **foundational example** of how to enable *agent-to-agent delegation* through a clean tool interface. Here are the key concepts and what you should focus on:

---

### 🧠 1. **Agent-as-a-Tool Pattern**

What stands out most is that **another agent is exposed as a callable tool** (`call_agent`). This allows any orchestrating agent to delegate part of a task without needing to understand the inner workings of the specialist agent. This is like plugging in a highly skilled subcontractor when needed.

**Focus on:**

* How this mirrors real-world collaboration
* How simple it becomes to *scale* capabilities

---

### 🧰 2. **Agent Registry Lookup**

```python
agent_registry = action_context.get_agent_registry()
agent_run = agent_registry.get_agent(agent_name)
```

This abstracts how agents are stored and accessed — you don't hardcode any dependencies. It's **dynamic**, modular, and decoupled. This also prevents tight coupling between agents.

**Focus on:**

* Registry = dynamic lookups → agents can be updated, swapped, or extended with no code changes elsewhere.

---

### 🧠 3. **Context Isolation**

```python
invoked_memory = Memory()
```

The called agent is given a **clean memory**. This ensures each task starts fresh — no risk of cross-contaminated state or prompts. It also prevents infinite recursion by *not* passing the registry itself again.

**Focus on:**

* Why isolated memory is crucial in multi-agent settings
* How this protects system stability

---

### 🛡️ 4. **Safe Execution with Fallbacks**

```python
try:
    ...
except Exception as e:
    return {"success": False, "error": str(e)}
```

Agent communication is wrapped in robust error handling. If the delegated agent fails, the caller can gracefully recover — a must-have for **reliable agent ecosystems**.

**Focus on:**

* Defensive coding and fault tolerance
* Return pattern: always structured, even on failure

---

### 🗃️ 5. **Uniform Output Schema**

The tool always returns:

```json
{
  "success": true/false,
  "agent": "agent_name",
  "result": "final content OR error"
}
```

This **standardizes** responses and makes the orchestrating agent's job easier when parsing the outcome.

**Focus on:**

* Inter-agent consistency
* Simple schemas for easy chaining

---

### In Summary

This code is the **bridge** between independent expert agents. You should think of it like a *router and translator* that allows modular agents to form flexible and intelligent workflows.





## 🤝 Agent-to-Agent Collaboration with `call_agent`

Let’s examine how the `call_agent` tool enables **inter-agent communication** within a multi-agent system. This tool follows a careful protocol to ensure proper agent invocation while maintaining safe boundaries.

When one agent needs to **leverage the expertise** of another, it simply calls this tool with:

* 🧠 The **target agent’s name**
* 📋 A **description of the task** to be performed

### 🔧 How It Works:

1. **Agent Lookup:**
   It retrieves the `agent_registry` from the `action_context`, which acts as a central directory for all available agents.

2. **Existence Check:**
   It verifies that the target agent exists in the system.

3. **Memory Isolation:**
   It creates a **fresh memory instance** for the new agent to ensure a clean execution — avoiding context contamination or recursion risks.

This design makes agent collaboration **safe, flexible, and modular** — allowing one agent to delegate work as easily as calling a tool.

---

## 🧭 Building a Meeting Scheduling System with Specialized Agents

Let’s explore what a **project management system** could look like where **two expert agents** collaborate:

### 🧑‍💼 Project Management Agent:

* Responsible for **deciding when meetings are needed** (based on tasks, deadlines, blockers).

### 🕒 Scheduling Specialist Agent:

* Handles the **logistics** of arranging meetings:

  * Checking calendars
  * Verifying availability
  * Creating event invites

By splitting responsibilities between strategic and operational roles, we mirror how **real-world teams function** — one agent focuses on *why* to meet, the other on *how* to meet.



In [ ]:
@register_tool()
def check_availability(
    action_context: ActionContext,
    attendees: List[str],
    start_date: str,
    end_date: str,
    duration_minutes: int,
    _calendar_api_key: str
) -> List[Dict]:
    """Find available time slots for all attendees."""
    return calendar_service.find_available_slots(...)

@register_tool()
def create_calendar_invite(
    action_context: ActionContext,
    title: str,
    description: str,
    start_time: str,
    duration_minutes: int,
    attendees: List[str],
    _calendar_api_key: str
) -> Dict:
    """Create and send a calendar invitation."""
    return calendar_service.create_event(...)

#====The scheduling specialist is focused entirely on finding times and creating meetings:

scheduler_agent = Agent(
    goals=[
        Goal(
            name="schedule_meetings",
            description="""Schedule meetings efficiently by:
            1. Finding times that work for all attendees
            2. Creating and sending calendar invites
            3. Handling any scheduling conflicts"""
        )
    ],
...
)




### **1. Tools Are Simple and Atomic**

* `check_availability` → Focused solely on finding time slots.
* `create_calendar_invite` → Responsible only for sending invites.
  These are **narrow, predictable operations** — very “Unix philosophy.”

**Why this matters:**
When debugging, you know exactly where to look. If times are wrong, you debug `check_availability`; if invites fail, you debug `create_calendar_invite`.

---

### **2. Separation of Concerns**

The tools do **only one job each** (finding time or creating events), while the `scheduler_agent` coordinates **how** these tools are used to fulfill higher-level goals.

**Takeaway:**
The agent doesn’t handle low-level logic — it focuses on **intent**:

> “Schedule meetings efficiently by checking times and creating invites.”

---

### **3. Use of Goals**

The `scheduler_agent` has **a single responsibility**:

```python
Goal(name="schedule_meetings", description="...")
```

This makes the agent’s purpose **crystal clear** and reduces complexity.

---

### **4. Hidden API Details**

Notice `_calendar_api_key` is passed through as a parameter. This shows the **agent/tool boundary** — the tool can securely access external services without exposing the complexity to the rest of the system.

---

### **5. Composition**

The **scheduler\_agent** isn’t trying to do everything — it’s *composed of smaller tools* that each know their job. This makes it easy to plug this agent into larger pipelines (like a project manager agent that calls `scheduler_agent`).



## Project Management Agent

This code defines the **project manager’s toolkit**, and what’s most important here is understanding how this agent **coordinates** work rather than executing it all directly. Here's what to focus on:

---

### 🔍 **1. Clear Ownership of Domain**

This agent operates entirely within the **project management domain**:

* It fetches project status
* It logs decisions
* And it can call other agents (like the scheduler)

**Key takeaway:** It doesn't know *how* to schedule meetings—it knows *when* meetings are needed.

---

### 🛠 **2. Role of `call_agent` Tool**

This is the **glue** between agents.

```python
@register_tool()
def call_agent(...): ...
```

Instead of reinventing scheduling logic, the project agent can simply say:

> “Hey, Scheduler Agent, please handle this.”

It’s **modular delegation** — the agent acts more like a manager than a technician.

---

### 🗂 **3. Narrow, Focused Tools**

Both `get_project_status()` and `update_project_log()` are:

* Readable
* Composable
* Easily testable

This keeps the tools predictable and easy to combine into workflows.

---

### 🧠 **4. Implicit Autonomy + Judgment**

When combined in a multi-agent system:

* The project agent makes **judgment calls** like *"a status update requires a sync-up"*
* The scheduler agent handles **logistics** like availability and invites

They form a **collaborative AI system**.

---

### ✅ Summary of What to Watch:

| Concept                         | Why It Matters                                            |
| ------------------------------- | --------------------------------------------------------- |
| `call_agent`                    | Enables agent-to-agent coordination                       |
| Tool boundaries                 | Each tool does one thing well — promotes reuse            |
| No overlap with scheduler tools | Shows **clear division of responsibility**                |
| Delegation model                | Lays foundation for building scalable multi-agent systems |





In [ ]:
@register_tool()
def get_project_status(
    action_context: ActionContext,
    project_id: str,
    _project_api_token: str
) -> Dict:
    """Retrieve current project status information."""
    return project_service.get_status(...)

@register_tool()
def update_project_log(
    action_context: ActionContext,
    entry_type: str,
    description: str,
    _project_api_token: str
) -> Dict:
    """Record an update in the project log."""
    return project_service.log_update(...)

@register_tool()
def call_agent(
    action_context: ActionContext,
    agent_name: str,
    task: str
) -> Dict:
    """Delegate to a specialist agent."""
    # Implementation as shown in previous tutorial




## 🧠 **Agent Declaration**

### **1. High-Level Goal Definition**

```python
Goal(
    name="project_oversight",
    description=...
)
```

This agent isn’t doing tactical tasks — it's **managing strategy**. Its job is to:

* Monitor project health
* Decide *when* a meeting is needed
* Delegate that work
* Log outcomes

📌 **Takeaway:** This is the **brains**, not the hands.

---

### **2. Clean Division of Responsibilities**

This is a textbook application of **separation of concerns**:

| Agent             | Responsibilities                                               |
| ----------------- | -------------------------------------------------------------- |
| `project_manager` | Interpret project status, decide when coordination is required |
| `scheduler_agent` | Handle the logistics of scheduling meetings                    |

This keeps both agents lean and testable.

---

### **3. Agent Composition via `call_agent`**

> “Delegating meeting scheduling to the 'scheduler\_agent'...”

This is where your multi-agent design comes to life. The `project_manager` doesn’t know how to send calendar invites — it knows *who* does.

This is **agent composition** — think microservices, but for LLMs.

---

### **4. Workflow as a Narrative**

The `description` reads like a playbook. Each step:

1. Pulls project data
2. Assesses needs
3. Calls another agent
4. Logs updates

This approach makes it:

* Easier to debug
* Easier to audit
* Easier to **extend** (e.g., add escalation logic)

---

### ✅ What You Should Take Away:

| Element                         | Why It Matters                                                |
| ------------------------------- | ------------------------------------------------------------- |
| **Goal-centric design**         | Keeps agents focused and understandable                       |
| **Delegation via `call_agent`** | Enables scalable, multi-agent coordination                    |
| **Narrative-based goals**       | Improves maintainability and traceability                     |
| **Division of labor**           | Mirrors real-world teamwork (PMs manage, schedulers schedule) |



In [ ]:
# The project management agent uses these tools to monitor progress and arrange meetings when needed:

# This division of responsibilities keeps each agent focused on its core competency:

# - The project manager understands project status and when meetings are needed
# - The scheduler excels at finding available times and managing calendar logistics
# - The call_agent tool allows seamless collaboration between them

project_manager = Agent(
    goals=[
        Goal(
            name="project_oversight",
            description="""Manage project progress by:
            1. Getting the current project status
            2. Identifying when meetings are needed if there are issues in the project status log
            3. Delegating meeting scheduling to the "scheduler_agent" to arrange the meeting
            4. Recording project updates and decisions"""
        )
    ],
    ...
)




# 🤝 Agent-to-Agent Communication: `call_agent` Tool

When agents collaborate, **clean communication** is critical. The `call_agent` tool provides a standardized, controlled way for one agent to invoke another — like handing off a task to a trusted colleague.

---

## 🧩 What Makes `call_agent` Powerful?

### ✅ **Memory Isolation**

Each agent runs with **its own memory instance** — so there's:

* No leakage between agents
* No accidental context carryover
* Full reproducibility and debugging

> Think: each agent gets a fresh notepad every time it's called.

---

### ✅ **Context Management**

The tool only passes the **necessary environment settings** to the sub-agent:

* ✅ API keys
* ✅ Shared tools
* ❌ Infinite recursion
* ❌ Confusing system state

This keeps the ecosystem **clean, bounded, and safe**.

---

### ✅ **Result Handling**

The system retrieves the **final memory output** of the sub-agent:

* Gives you just the result (not the whole transcript)
* Enables clean composition (like returning from a function)
* Makes it easy to pipe outputs between agents

---

# 🧠 Registering Agents: The Agent Directory

To support multi-agent coordination, you need a **registry** to track who can do what:

```python
class AgentRegistry:
    def __init__(self):
        self.agents = {}

    def register_agent(self, name: str, run_function: callable):
        """Register an agent's run function."""
        self.agents[name] = run_function

    def get_agent(self, name: str) -> callable:
        """Get an agent's run function by name."""
        return self.agents.get(name)
```

This lets any agent say:

> “I need help. Let me check the directory and call the right specialist.”

---

### 🏗 System Setup Example

```python
# Initialize registry
registry = AgentRegistry()

# Register your agents
registry.register_agent("scheduler_agent", scheduler_agent.run)

# Inject registry into shared context
action_context = ActionContext({
    'agent_registry': registry,
    # Add API keys, env objects, logs, etc.
})
```

Now your agents can **find and delegate** tasks without hardcoding references.

---

## 🧠 TL;DR

| Feature                | Benefit                                           |
| ---------------------- | ------------------------------------------------- |
| 🔐 Memory Isolation    | Prevents data bleeding between agents             |
| 🎯 Scoped Context      | Only passes what’s necessary to sub-agents        |
| 📤 Clean Return Values | Makes it easy to use results in further workflows |
| 📇 Agent Registry      | Enables dynamic, scalable coordination            |

Let me know if you want to see how the `call_agent` tool is implemented internally — it’s a great way to learn about delegation patterns.


In [ ]:
# Registering Agents

# To make this system work, we need to register our agents in the registry:

class AgentRegistry:
    def __init__(self):
        self.agents = {}

    def register_agent(self, name: str, run_function: callable):
        """Register an agent's run function."""
        self.agents[name] = run_function

    def get_agent(self, name: str) -> callable:
        """Get an agent's run function by name."""
        return self.agents.get(name)

# When setting up the system
registry = AgentRegistry()
registry.register_agent("scheduler_agent", scheduler_agent.run)

# Include registry in action context
action_context = ActionContext({
    'agent_registry': registry,
    # Other shared resources...
})
